In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import yaml
import re
import time
from tqdm import tqdm_notebook as tqdm
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
import matplotlib.pyplot as plt
np.random.seed(444)
import matplotlib.pyplot as plt
import xgboost
import math
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [2]:
file_name = "MOSCOW_SALE_for_preprocessing.csv"
df = pd.read_csv(file_name)
df.head()

,url,geometry,deal_type,title,address,city,region,district,street,house,...,building_info,district_info,infrastructure,technical_details,add_infrastructure,POIs_50,POIs_100,POIs_300,OFD,CELL
0,http://www.cian.ru/sale/commercial/10046273/,POINT (55.81026800000001 37.606334),sale,"Офис (B), 2 500 м²","Москва, СВАО, р-н Бутырский, Огородный проезд,...",Москва,СВАО,р-н Бутырский,Огородный проезд,5С1,...,{},{},{},{},{'add_infrastructure': ['Приточная вентиляция'...,{'fuel': 1},{'fuel': 1},"{'fuel': 2, 'bench': 1, 'car_wash': 1, 'bicycl...","{0: 1760.0, 1: 260.0, 4: 600.0, 7: 208.0, 8: 2...","{'00:00': 1214, '00:30': 1217, '01:00': 1208, ..."
1,http://www.cian.ru/sale/commercial/10046393/,POINT (55.80954000000001 37.605975),sale,"Офис (B), 3 500 м²","Москва, СВАО, р-н Бутырский, Огородный проезд,...",Москва,СВАО,р-н Бутырский,Огородный проезд,5с6,...,{},{},{},{},{'add_infrastructure': ['Приточная вентиляция'...,{},{},"{'fuel': 2, 'parking': 1, 'fast_food': 1}","{0: 1760.0, 1: 260.0, 4: 600.0, 7: 208.0, 8: 2...","{'00:00': 922, '00:30': 926, '01:00': 925, '01..."
2,http://www.cian.ru/sale/commercial/10046423/,POINT (55.809064 37.60673),sale,"Офис (B), 4 500 м²","Москва, СВАО, р-н Бутырский, Огородный проезд,...",Москва,СВАО,р-н Бутырский,Огородный проезд,5С3,...,{},{},{},{},{'add_infrastructure': ['Приточная вентиляция'...,{},{},"{'fuel': 2, 'parking': 2, 'car_wash': 1, 'fast...","{0: 1760.0, 1: 260.0, 4: 600.0, 7: 208.0, 8: 2...","{'00:00': 922, '00:30': 926, '01:00': 925, '01..."
3,http://www.cian.ru/sale/commercial/10046453/,POINT (55.809914 37.607035),sale,"Офис (B), 6 000 м²","Москва, СВАО, р-н Бутырский, Огородный проезд,...",Москва,СВАО,р-н Бутырский,Огородный проезд,5С2,...,{},{},{},{},{'add_infrastructure': ['Приточная вентиляция'...,{},{'fuel': 1},"{'fuel': 2, 'bench': 1, 'car_wash': 1, 'parkin...","{0: 1760.0, 1: 260.0, 4: 600.0, 7: 208.0, 8: 2...","{'00:00': 1724, '00:30': 1735, '01:00': 1727, ..."
4,http://www.cian.ru/sale/commercial/10046543/,POINT (55.81026800000001 37.606334),sale,"Офис (B), 7 000 м²","Москва, СВАО, р-н Бутырский, Огородный проезд,...",Москва,СВАО,р-н Бутырский,Огородный проезд,5С1,...,{},{},{},{},{'add_infrastructure': ['Приточная вентиляция'...,{'fuel': 1},{'fuel': 1},"{'fuel': 2, 'bench': 1, 'car_wash': 1, 'bicycl...","{0: 1760.0, 1: 260.0, 4: 600.0, 7: 208.0, 8: 2...","{'00:00': 1214, '00:30': 1217, '01:00': 1208, ..."


In [3]:
df['description_text'] = df.description.apply(lambda x:x.split('}, ')[1].replace(')','').replace("'",''))
df.description = df.description.apply(lambda x:'{'+x.split('}, ')[0].replace('({', '')+'}')

In [4]:
columns = ['geometry','deal_type','title','address','city','region','district','street', 
           'house','description_text','metro','description','rules','add_info','building_info', 
           'district_info', 'infrastructure','technical_details', 'add_infrastructure', 'POIs_50', 
           'POIs_100','POIs_300', 'OFD', 'CELL']
df = df[columns]

In [5]:
df.metro = df.metro.apply(lambda x:yaml.load(x))
df.description = df.description.apply(lambda x:yaml.load(x))
df.rules = df.rules.apply(lambda x:yaml.load(x))
df.building_info = df.building_info.apply(lambda x:yaml.load(x))
df.district_info = df.district_info.apply(lambda x:yaml.load(x))
df.infrastructure = df.infrastructure.apply(lambda x:yaml.load(x))
df.technical_details = df.technical_details.apply(lambda x:yaml.load(x))
df.add_infrastructure = df.add_infrastructure.apply(lambda x:yaml.load(x))

In [6]:
df.add_info = df.add_info.apply(lambda x:yaml.load(x))

# Для чистки по "тяжелым" строкам
def rel(data):
    if "" in data:
        rez = 1
    else:
        rez = 0
    return rez
    
df['marker'] = df.add_info.apply(rel)
df = df.drop(df[df.marker ==1].index)
df = df.drop(['marker'],axis = 1)

In [7]:
df.POIs_50 = df.POIs_50.apply(lambda x:yaml.load(x))
df.POIs_100 = df.POIs_100.apply(lambda x:yaml.load(x))
df.POIs_300 = df.POIs_300.apply(lambda x:yaml.load(x))
df.OFD = df.OFD.apply(lambda x:yaml.load(x))
df.CELL = df.CELL.apply(lambda x:yaml.load(x))

In [8]:
df.to_pickle('./Mos_sale.pkl')